# The Impact of Subsampling

How much worse does performance get when we subsample the N-CMAPSS dataset?

In [43]:
import torch
import torch.nn as nn
torch.multiprocessing.set_sharing_strategy('file_system')
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import DataUtils
import Masking

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Loading (Subsampled)

In [45]:
batch_size = 64
trainloader, testloader = DataUtils.get_ncmapss_dataloaders(1, n_timesteps=10, batch=batch_size, workers=8, subsampled=True)

## Defining an RUL Transformer Model

In [46]:
class RULTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.prediction_window = 1 # only estimates the next point
        self.input_len = 10 # timesteps of 10
        self.n_cols = 21 # 46 columns present in the data
        self.embed_dim = 64 # project to a 64-dimensional space
        
        self.input_projection = nn.Linear(self.input_len*self.n_cols, self.embed_dim) 
        self.positional_embed = nn.Parameter(torch.randn(self.embed_dim)) # learned positional embeddings
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.embed_dim, nhead=4, activation="gelu", dim_feedforward=64)
        self.transformer_blocks = nn.TransformerEncoder(self.encoder_layer, num_layers=1)
        
        self.rul_head = nn.Sequential(nn.Linear(self.embed_dim, self.prediction_window)) # linear rul prediction head 
    
    def forward(self, x):
        x = x.flatten(1)
        z = self.input_projection(x)
        z = z + self.positional_embed
        z = self.transformer_blocks(z)
        z = self.rul_head(z)
        
        return z.squeeze(1)

## Training on Clean Data

In [47]:
objective = nn.MSELoss()

rul_tran = RULTransformer().to(device)

columns_kept = [True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               False, False, False, False, False, False, False, False, False, False, False, False, False, False,
               False, False, False, False, False, False, False, False, False, False,
               True, True, True, False]

lr = 2e-5
n_epochs = 13
optim = torch.optim.Adam(rul_tran.parameters(), lr=lr)
losses = []

for n in range(n_epochs):
    counter = 0
    for i, (X, y) in enumerate(tqdm(trainloader)):
        X = X.to(device)[:, :, columns_kept]
        y = y.to(device)
        optim.zero_grad()
        yhat = rul_tran(X.float())
        loss = objective(yhat, y.float().squeeze(1))
        loss.backward()
        losses.append(loss.cpu().detach().numpy())
        optim.step()
        counter += 1
        
    print("Epoch:", n+1, "Loss:",np.mean(losses[-counter:][0]))
    
    test_mses = []
    yhats = []
    ys = []
    with torch.no_grad():
        for i, (X, y) in enumerate(tqdm(testloader)):
            yhat = rul_tran(X.float().to(device)[:, :, columns_kept])
            yhats.append(yhat.cpu())
            ys.append(y.cpu())
            test_mse = objective(yhat, y.float().to(device))
            test_mses.append(test_mse.item())
    print("Test MSE:", np.mean(test_mses))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:14<00:00, 103.06it/s]


Epoch: 1 Loss: 2359.6848


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4274/4274 [00:40<00:00, 106.59it/s]


Test MSE: 1169.1570902989447


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:07<00:00, 113.05it/s]


...

Test MSE: 17.596673233017054


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7667/7667 [01:10<00:00, 108.70it/s]


Epoch: 13 Loss: 0.2965749


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4274/4274 [00:40<00:00, 106.09it/s]

Test MSE: 18.105958307938536


## Loading Full Data

In [48]:
batch_size = 64
full_trainloader, full_testloader = DataUtils.get_ncmapss_dataloaders(1, n_timesteps=10, batch=batch_size, workers=8, subsampled=False)

In [49]:
test_mses = []
yhats = []
ys = []
with torch.no_grad():
    for i, (X, y) in enumerate(tqdm(full_testloader)):
        yhat = rul_tran(X.float().to(device)[:, :, columns_kept])
        yhats.append(yhat.cpu())
        ys.append(y.cpu())
        test_mse = objective(yhat, y.float().to(device))
        test_mses.append(test_mse.item())
print("Test MSE:", np.mean(test_mses))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42738/42738 [13:34<00:00, 52.46it/s]

Test MSE: 16.52309666749518


In [50]:
print("Test RMSE: {}".format(np.sqrt(np.mean(test_mses))))

Test RMSE: 4.064861211344759


## Training on Full Dataset

In [39]:
objective = nn.MSELoss()

full_rul_tran = RULTransformer().to(device)

columns_kept = [True, True, True, True,
               True, True, True, True, True, True, True, True, True, True, True, True, True, True,
               False, False, False, False, False, False, False, False, False, False, False, False, False, False,
               False, False, False, False, False, False, False, False, False, False,
               True, True, True, False]

lr = 2e-5
n_epochs = 13
optim = torch.optim.Adam(full_rul_tran.parameters(), lr=lr)
losses = []

for n in range(n_epochs):
    counter = 0
    for i, (X, y) in enumerate(tqdm(full_trainloader)):
        X = X.to(device)[:, :, columns_kept]
        y = y.to(device)
        optim.zero_grad()
        yhat = full_rul_tran(X.float())
        loss = objective(yhat, y.float().squeeze(1))
        loss.backward()
        losses.append(loss.cpu().detach().numpy())
        optim.step()
        counter += 1
        
    print("Epoch:", n+1, "Loss:",np.mean(losses[-counter:][0]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76667/76667 [25:38<00:00, 49.84it/s]


Epoch: 1 Loss: 2659.6562


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76667/76667 [23:28<00:00, 54.44it/s]


Epoch: 2 Loss: 0.60821044


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76667/76667 [23:37<00:00, 54.09it/s]


...

Epoch: 11 Loss: 0.08111506


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76667/76667 [23:23<00:00, 54.63it/s]


Epoch: 12 Loss: 0.0376716


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76667/76667 [22:32<00:00, 56.68it/s]

Epoch: 13 Loss: 0.041961975


In [40]:
test_mses = []
yhats = []
ys = []
with torch.no_grad():
    for i, (X, y) in enumerate(tqdm(full_testloader)):
        yhat = full_rul_tran(X.float().to(device)[:, :, columns_kept])
        yhats.append(yhat.cpu())
        ys.append(y.cpu())
        test_mse = objective(yhat, y.float().to(device))
        test_mses.append(test_mse.item())
print("Test MSE:", np.mean(test_mses))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42738/42738 [13:07<00:00, 54.27it/s]

Test MSE: 353.37011108187033


In [41]:
print("Test RMSE: {}".format(np.sqrt(np.mean(test_mses))))

Test RMSE: 18.79814116028152


In [42]:
np.save('./subsampling.npy', np.sqrt(np.mean(test_mses)))

We see a RMSE of around 18 here. This is too low - there probably needs to be some hyperparameter optimization. However, this would take hundreds hours and is probably not worth it. Instead, we can see results from the literature. For example, another paper that used Transformers on DS01 achieved 4.54 RMSE. (Notably, they subsampled too by averaging the first half and second half from each cycle.)

  Li, Xinyao, Jingjing Li, Lin Zuo, Lei Zhu, and Heng Tao Shen. “Domain Adaptive Remaining Useful Life Prediction With Transformer.” IEEE Transactions on Instrumentation and Measurement 71 (2022): 1–13. https://doi.org/10.1109/TIM.2022.3200667.

In other words, our subsampling does lose a lot of performance.